In [1]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

TAREA: Proponer un demostrador que capture las imágenes de la cámara, y les permita exhibir lo aprendido en estas dos prácticas ante quienes no cursen la asignatura :). Es por ello que además de poder mostrar la imagen original de la webcam, incluya al menos dos usos diferentes de aplicar las funciones de OpenCV trabajadas hasta ahora.

Uso número 1: Aplicamos el algoritmo Canny para la detección de bordes en una imagen.

In [2]:
# Capturamos el video desde la webcam

cam = cv2.VideoCapture(0)

while True:

    #Capturamos fotograma por fotograma
    ret, frame = cam.read()

    if ret:

        # Valor para el umbralizado posterior
        threshold = 150

        frame_to_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Aplicar desenfoque para reducir el ruido y evitar detecciones falsas de bordes
        frame_blurred_canny = cv2.GaussianBlur(frame_to_gray, (5, 5), 1.4)
        frame_blurred_sobel = cv2.GaussianBlur(frame_to_gray, (3, 3), 0)

        # Detectar bordes utilizando el algoritmo de Canny, ajustando el umbral superior e inferior para mayor o menor sensibilidad
        # Threshold1: bordes fuertes; Threshold2: bordes débiles.
        frame_canny = cv2.Canny(frame_blurred_canny, threshold1=30, threshold2=90)

        # Detección de bordes en ambas direcciones (X e Y), y luego combinando, utilizando el algoritmo de Sobel
        frame_sobel_x = cv2.Sobel(frame_blurred_sobel, cv2.CV_64F, 1, 0)    # X
        frame_sobel_y = cv2.Sobel(frame_blurred_sobel, cv2.CV_64F, 0, 1)    # Y
        frame_sobel = cv2.add(frame_sobel_x, frame_sobel_y)                 # Combinamos X e Y

        # Calcular la magnitud del gradiente
        magnitud = np.sqrt(frame_sobel_x**2 + frame_sobel_y**2)
        # Convertir la magnitud a uint8
        magnitud = np.uint8(magnitud / np.max(magnitud) * 255)

        # Opcional: aplicar umbral para resaltar bordes más definidos
        _, frame_sobel = cv2.threshold(magnitud, 70, 255, cv2.THRESH_BINARY)

        # Umbralización binaria del video
        _, frame_threshold = cv2.threshold(frame_to_gray, threshold, 255, cv2.THRESH_BINARY)


        # Mostrar el fotograma original y el de bordes
        cv2.imshow('Video original', frame)
        cv2.imshow('Deteccion de bordes Canny', frame_canny)
        cv2.imshow('Deteccion de bordes Sobel', frame_sobel)
        cv2.imshow('Umbralizado', frame_threshold)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break


# Libera el objeto de captura
cam.release()
# Destruye ventanas
cv2.destroyAllWindows()


La última tarea consiste en la detección de caras y su respectivo pixelado en tiempo real, a través de la página web.

In [22]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cam = cv2.VideoCapture(0)

def pixelate(image, blocks=10):
    # Obtiene las dimensiones de la imagen
    height, width = image.shape[:2]

    # Calcula el tamaño de cada bloque
    x_steps = width // blocks
    y_steps = height // blocks

    # Itera sobre la imagen en bloques
    for y in range(0, height, y_steps):
        for x in range(0, width, x_steps):
            # Define la región del bloque
            roi = image[y:y+y_steps, x:x+x_steps]

            # Calcula el color medio del bloque
            (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]

            # Asigna el color medio al bloque
            cv2.rectangle(image, (x, y), (x + x_steps, y + y_steps), (B, G, R), -1)

    return image

while True:
    
    # Capturamos frame a frame
    ret, frame = cam.read()

    if ret:

        frame_to_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detección de la cara detectando rectámngilos que delimitan el objeto especificado (cara ya que viene del objeto face_cascade )
        face_detection = face_cascade.detectMultiScale(frame_to_gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in face_detection:
            face_roi = frame[y:y+h, x:x+w]
            pixelated_face = pixelate(face_roi, blocks=15)
            frame[y:y+h, x:x+w] = pixelated_face


    cv2.imshow('Pixelado en Tiempo Real', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

cam.release()
cv2.destroyAllWindows()

    

